<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')

def get_combined_signal(stock):
    # Fetch monthly stock data
    data = yf.download(stock + '.NS', period='5y', interval='1mo')

    data['Symbol'] = stock
    # Calculate 5-period and 13-period EMA
    data['5_EMA'] = data['Close'].ewm(span=5, adjust=False).mean()
    data['13_EMA'] = data['Close'].ewm(span=13, adjust=False).mean()

    # Calculate MACD (8-period EMA - 21-period EMA) and Signal line (9-period EMA of MACD)
    data['26_EMA'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['12_EMA'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['MACD'] = data['12_EMA'] - data['26_EMA']
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Determine if MACD is up or down
    data['MACD_Up'] = data['MACD'] > data['Signal_Line']
    data['MACD_Down'] = data['MACD'] < data['Signal_Line']

    # Determine combined signal
    data['Momentum'] = 'NA'
    data.loc[(data['5_EMA'] > data['13_EMA']) & (data['MACD_Up']), 'Momentum'] = 'UP'
    data.loc[(data['5_EMA'] < data['13_EMA']) & (data['MACD_Down']), 'Momentum'] = 'DOWN'

    return data[['Symbol', 'Close', 'Momentum']]

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')
  return mypps_df

mypps_df = get_mypps_df()

exclude = ['CADILAHC','MMTC', 'SCHAEFFLER ']

stock_ids = mypps_df[~mypps_df['Symbol'].isin(exclude) ]['Symbol'].unique()

df_prec_dev = pd.DataFrame()
for stock_id in stock_ids:
  df_pos = get_combined_signal(stock_id)

  df_pos['Prev_Momentum'] = df_pos['Momentum'].shift(1)
  df_pos = df_pos[df_pos['Momentum'] != df_pos['Prev_Momentum']]
  df_pos['Signal']= 'Sell'
  df_pos.loc[df_pos['Momentum'] == 'UP', 'Signal'] = 'Buy'
  df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
  df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
  df_pos.drop(['Prev_Momentum', 'Prev_Signal'], axis=1, inplace=True)
  df_pos['Close'] = round(df_pos['Close'],0)
  df_pos = df_pos.reset_index()
  df_prec_dev = pd.concat([df_prec_dev, df_pos.tail(1)], ignore_index = True)

df_prec_dev = df_prec_dev.sort_values(by = 'Date', ascending=True)
df_prec_dev.index = df_prec_dev['Date']

df_prec_dev.drop(['Date'], axis=1, inplace=True)
df_tmp = df_prec_dev.drop(['Close'], axis=1)
df_tmp.to_csv('/content/drive/My Drive/data/stocks/myProspects-Momentum.csv', index=False)
df_prec_dev.drop(['Momentum'], axis=1, inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [5]:
df_prec_dev[df_prec_dev['Signal'] == 'Sell'].tail(20)

,Symbol,Close,Signal
Date,,,
2024-03-01,KANSAINER,262.0,Sell
2024-03-01,LTIM,4938.0,Sell
2024-04-01,TASTYBITE,11887.0,Sell
2024-04-01,BERGEPAINT,509.0,Sell
2024-04-01,HDFCLIFE,584.0,Sell
2024-05-01,PGHH,15971.0,Sell
2024-05-01,VAIBHAVGBL,338.0,Sell
2024-05-01,TRIDENT,36.0,Sell
2024-05-01,EQUITASBNK,93.0,Sell


In [6]:
df_prec_dev[df_prec_dev['Signal'] == 'Buy'].tail(20)

,Symbol,Close,Signal
Date,,,
2024-05-01,EMAMILTD,626.0,Buy
2024-05-01,CROMPTON,392.0,Buy
2024-06-01,ASIANTILES,72.0,Buy
2024-06-01,AKZOINDIA,2835.0,Buy
2024-06-01,AAVAS,1853.0,Buy
2024-06-01,MARICO,613.0,Buy
2024-06-01,WHIRLPOOL,1998.0,Buy
2024-06-01,DABUR,601.0,Buy
2024-06-01,PRAJIND,729.0,Buy
